In [1]:
import pandas as pd
import numpy as np
import quandl

quandl.ApiConfig.api_key = "zFtyXTJz6DTqqdvEy2gi"

Stock Data (DBA and WEAT not available from here)

In [ ]:
data = quandl.get_table('WIKI/PRICES', qopts = { 'columns': ['ticker', 'date', 'close'] }, ticker = ['DBA'], date = { 'gte': '2012-01-01', 'lte': '2018-04-13' })

### Futures: Continuous contracts (W# - contract # ahead)

In [ ]:
#continuous futures W1 is front month
#W1 may18 prices start 3/15/18..futures trade until biz day before 15th of settle month
data = quandl.get('CHRIS/CME_W7')

In [ ]:
data.to_csv('CME_W7.csv')

In [ ]:
data.head()

In [ ]:
data.iloc[14780:,]

### Futures: Individual contracts (all data)

In [ ]:
#individual contract (July 2018)
#data = quandl.get('CME/WN2018')
#data = quandl.get('CME/WK2018')  #may2018'
data = quandl.get('CME/WH2018')  #mar2018


In [ ]:
#data.iloc[635:,[3]]

### Combining the Financial data sets by Date

In [65]:
# dat = pd.read_csv("Data/CME_W4.csv")
# dat = dat.rename(index=str, columns={'Volume': 'Vol', 'Previous Day Open Interest': 'OpIntPvD'})
# print(dat.head())
# print(len(dat))
# dat.dtypes

# i = 1
# cols = ['Date', 'Open', 'High', 'Low', 'Last', 'Vol', 'OpInt']
# dat = dat[cols]
# cols2 = ['W'+str(i) + "_" + x for x in cols[1:]]
# cols2.insert(0,'Date')
# dat.columns = cols2
# dat.head()

### Combine all the Futures Contract Data (W1-3 since '59, W4-5 all since '05)

In [83]:
def fixDate(inDate):
    lst = inDate.split('/')
    if lst[2] < '20':
        lst[2] = '20'+lst[2]
    else:
        lst[2] = '19'+lst[2]
    return '/'.join(lst)
    
import glob

cols = ['Date', 'Open', 'High', 'Low', 'Last', 'Vol', 'OpInt']

#get futures data files, sort by 13th char (digit)
filenamesFUT = glob.glob('./Data/CME_W*.csv')
filenamesFUT = sorted(filenamesFUT, key=lambda x: x[12])

#for each file, subset/clean columns, join to W1 by date
for file in filenamesFUT:
    lst = file.split('W')
    cNum = int(lst[1][:1])
    df = pd.read_csv(file)
    
    #W1-3 have date without century, add full year
    if cNum <= 3:
        df['Date'] = df['Date'].apply(fixDate)
    df['Date'] = pd.to_datetime(df['Date'])
    
    df = df.rename(index=str, columns={'Volume': 'Vol', 'Previous Day Open Interest': 'OpInt'})
    df = df[cols]
    colsTxt = ['W'+str(cNum) + "_" + x for x in cols[1:]]
    colsTxt.insert(0, 'Date')
    df.columns = colsTxt

    if cNum == 1:
        dat = df
    else:
        dat = dat.merge(df, how='left', on='Date')



### Add Equity data

In [84]:
filenames = glob.glob('./Data/*.csv')
filenamesEQ = [x for x in filenames if x not in filenamesFUT]

colsEQ = ['Date','Close']

for file in filenamesEQ:
    if 'financialALL' not in file:
        df = pd.read_csv(file, parse_dates=[0])
        df = df[colsEQ]
    
        #extract ticker
        ticker = file.split('/')[2].split('.')[0]

        df.columns = ['Date', ticker]
        print(df.dtypes)
        print()
        #merge with futures data
        dat = dat.merge(df, how='left', on='Date')

Date    datetime64[ns]
ADM            float64
dtype: object

Date    datetime64[ns]
BG             float64
dtype: object

Date    datetime64[ns]
WEAT           float64
dtype: object

Date    datetime64[ns]
DBA            float64
dtype: object



In [85]:
dat.columns

Index(['Date', 'W1_Open', 'W1_High', 'W1_Low', 'W1_Last', 'W1_Vol', 'W1_OpInt',
       'W2_Open', 'W2_High', 'W2_Low', 'W2_Last', 'W2_Vol', 'W2_OpInt',
       'W3_Open', 'W3_High', 'W3_Low', 'W3_Last', 'W3_Vol', 'W3_OpInt',
       'W4_Open', 'W4_High', 'W4_Low', 'W4_Last', 'W4_Vol', 'W4_OpInt',
       'W5_Open', 'W5_High', 'W5_Low', 'W5_Last', 'W5_Vol', 'W5_OpInt', 'ADM',
       'BG', 'WEAT', 'DBA'],
      dtype='object')

In [86]:
filenamesEQ

['./Data/ADM.csv',
 './Data/BG.csv',
 './Data/WEAT.csv',
 './Data/financialALL.csv',
 './Data/DBA.csv']

In [87]:
dat.to_csv("./Data/financialALL.csv")

In [88]:
dfW = pd.read_csv("./Data/pivoted_daily.csv", )
dfW.head()

/Users/Arobster/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,

,Unnamed: 0,max_temp_f,max_temp_f.1,max_temp_f.2,max_temp_f.3,max_temp_f.4,max_temp_f.5,max_temp_f.6,max_temp_f.7,max_temp_f.8,...,snowd_in.83,snowd_in.84,snowd_in.85,snowd_in.86,snowd_in.87,snowd_in.88,snowd_in.89,snowd_in.90,snowd_in.91,snowd_in.92
0,station,1F0,AAO,ADM,AFK,AIA,ANW,AUH,BBW,BFF,...,SWO,TIF,TOP,TQE,TQH,TUL,VTN,WDG,WLD,WWR
1,day,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2005-01-31,43.0,35.96,45.0,37.4,44.6,42.8,37.4,44.96,44.96,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
3,2005-02-01,43.0,37.04,45.0,39.2,44.96,46.4,37.4,42.98,44.96,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
4,2005-02-02,43.0,44.06,45.0,46.4,48.92,53.6,44.6,51.98,51.8,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
